In [1]:
from data_loader import DataLoader
from models import MODELS
from misc.evaluation_metrics import *

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3)

import json
import warnings
warnings.filterwarnings("ignore")

dl = DataLoader()
info = dl.get_info()

info

,name,frequency,description,source,path
0,Electricity Transformer Data - 15 min,fifteen minutes,...,https://github.com/zhouhaoyi/ETDataset,data/electricity_transformer_15min.csv
1,Metro Interstate Human Traffic Volume,hour,Hourly Interstate 94 Westbound traffic volume ...,https://archive.ics.uci.edu/ml/datasets/Metro+...,data/Metro_Interstate_Traffic_Volume.csv
2,Beijing-Guanyuan Air-Quality,hour,This hourly data set considers 6 main air poll...,https://archive.ics.uci.edu/ml/datasets/Beijin...,data/beijing_air_quality_Data_Guanyuan_2013030...
3,Solar Generation - EnerjiSA,hour,Solar generation data of unlicenced solar cen...,https://www.kaggle.com/competitions/enerjisa-e...,data/enerjisa_solar_generation.csv


In [10]:
index = 1

dataset_name = info.name[index]
print(dataset_name)

df = dl.load(index)


if dataset_name == "Electricity Transformer Data - 15 min":
    (X_train, y_train, X_test, y_test), scaler = dl.prepare_dataset(23*4,1)

elif dataset_name == "Metro Interstate Human Traffic Volume":
    (X_train, y_train, X_test, y_test), scaler = dl.prepare_dataset(24*5,12)

elif dataset_name == "Beijing-Guanyuan Air-Quality":
    (X_train, y_train, X_test, y_test), scaler = dl.prepare_dataset(23,1)

elif dataset_name == "Solar Generation - EnerjiSA":
    (X_train, y_train, X_test, y_test), scaler = dl.prepare_dataset(23,1)

print(X_train.shape)

Metro Interstate Human Traffic Volume
                      value
date_time                  
2012-10-02 09:00:00  5545.0
2012-10-02 10:00:00  4516.0
2012-10-02 11:00:00  4767.0
2012-10-02 12:00:00  5026.0
2012-10-02 13:00:00  4918.0
...                     ...
2018-09-30 19:00:00  3543.0
2018-09-30 20:00:00  2781.0
2018-09-30 21:00:00  2159.0
2018-09-30 22:00:00  1450.0
2018-09-30 23:00:00   954.0

[48204 rows x 1 columns]
                        value
date_time                    
2012-10-02 09:00:00  0.761676
2012-10-02 10:00:00  0.620330
2012-10-02 11:00:00  0.654808
2012-10-02 12:00:00  0.690385
2012-10-02 13:00:00  0.675549
...                       ...
2018-09-30 19:00:00  0.486676
2018-09-30 20:00:00  0.382005
2018-09-30 21:00:00  0.296566
2018-09-30 22:00:00  0.199176
2018-09-30 23:00:00  0.131044

[48204 rows x 1 columns]
(33650, 120, 1)


In [22]:
list(MODELS.keys())

model = MODELS[list(MODELS.keys())[0]]
model = model((X_train.shape[1],X_train.shape[2]), 12)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 32)                4352      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 12)                396       
                                                                 
Total params: 4,748
Trainable params: 4,748
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 32)                4352      
                                                                 
 dropout_3 (Dropout)         (

In [23]:
model.compile(optimizer="adam",loss="MSE")
model.fit(
    X_train,
    y_train,
    validation_split=0.25, 
    epochs=50,
    callbacks=[early_stopping])

Epoch 1/50
789/789 [==============================] - 7s 8ms/step - loss: 0.0717 - val_loss: 0.0439
Epoch 2/50
789/789 [==============================] - 6s 7ms/step - loss: 0.0526 - val_loss: 0.0401
Epoch 3/50
789/789 [==============================] - 6s 7ms/step - loss: 0.0483 - val_loss: 0.0360
Epoch 4/50
789/789 [==============================] - 6s 7ms/step - loss: 0.0445 - val_loss: 0.0322
Epoch 5/50
789/789 [==============================] - 6s 7ms/step - loss: 0.0415 - val_loss: 0.0309
Epoch 6/50
789/789 [==============================] - 6s 7ms/step - loss: 0.0397 - val_loss: 0.0311
Epoch 7/50
789/789 [==============================] - 6s 7ms/step - loss: 0.0384 - val_loss: 0.0296
Epoch 8/50
789/789 [==============================] - 6s 7ms/step - loss: 0.0376 - val_loss: 0.0286
Epoch 9/50
789/789 [==============================] - 6s 7ms/step - loss: 0.0369 - val_loss: 0.0296
Epoch 10/50
789/789 [==============================] - 6s 7ms/step - loss: 0.0362 - val_loss: 0.0289

In [35]:
#Prediction Info
pred = model.predict(X_test)
# pred = scaler.inverse_transform(pred).reshape(1,-1)[0]
# y_test_inv = scaler.inverse_transform(y_test).reshape(1,-1)[0]

pred = scaler.inverse_transform(pred)
y_test_inv = scaler.inverse_transform(y_test)

print(
    " R2 : ", round(r2_score(y_test_inv,pred),2), "\n", \
    "MAE : ", round(MAE(y_test_inv,pred),2), "\n", \
    "RMSE : ", round(RMSE(y_test_inv,pred),2), "\n", \
    "MSE : ", round(MSE(y_test_inv,pred),2), "\n", \
    "MAPE : ", round(MAPE(y_test_inv,pred),2), "\n", \
    "SMAPE : ", round(SMAPE(y_test_inv,pred),2), "\n", \
    "MDAPE : ", round(MDAPE(y_test_inv,pred),2)
)

 R2 :  0.63 
 MAE :  850.01 
 RMSE :  1196.71 
 MSE :  1432110.89 
 MAPE :  68.25 
 SMAPE :  37.28 
 MDAPE :  19.9


In [45]:
import plotly.graph_objects as go

fig = go.Figure(go.Line(y=y_test_inv[:,3]))
fig.add_trace(go.Line(y=pred[:,3]))
fig